# api 크롤링

기존의 크롤링 방식은 

1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium)

혹은 

2. 파이썬 자체적으로 호출을 요청해서(requests)

데이터를 가지고 오는 방식이었습니다.

이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.

따라서 서비스 제공자 측에서는 서버 부하를 줄이고, 사용자에게 맞춤형의 데이터를 제공하기 위해 api서버를 운영합니다.

api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.)

필요없는 동영상 자료나 그림 자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.


api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.

In [1]:
# 사이트에 자료 요청
import urllib.request
#json 데이터 핸들링
import json
# DataFrame 자료형 활용
import pandas as pd 
# json 데이터를 pandas DataFrame으로 변환
from pandas.io.json import json_normalize

# 영진위 api 신청

https://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

1. 접속 후 가입 및 로그인 

2. 키발급 탭에서 키 발급하기 

3. 요청 api종류 보고 요청양식 및 데이터 확인하기 

In [2]:
api_key = "5c6ac9b9844cd60b13a5ea12c2eede98" # 발급받은 키만 복붙하세요.
request_date = "20220802" # 조회날짜를  xxxxyydd 20220802 형식으로 넣어주세요.
url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
print(url)

https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5c6ac9b9844cd60b13a5ea12c2eede98&targetDt=20220802


# json데이터를 팬더스 데이터프레임으로 변환

api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다.

쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데

파이썬의 딕셔너리와 거의 같다고 보시면 됩니다.

따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상

딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [3]:
# 위의 url 변수를 이용해 데이터를 요청합니다.
r = urllib.request.urlopen(url)

# 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
json_raw =r.read()

In [4]:
# b'~~~~~~~'로 시작하는 이 데이터는 파이썬에서 byte자료형으로 처리되고
# 데이터프레임으로는 바로 변환이 불가능합니다.
json_raw

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20220802~20220802","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20209343","movieNm":"\xed\x95\x9c\xec\x82\xb0: \xec\x9a\xa9\xec\x9d\x98 \xec\xb6\x9c\xed\x98\x84","openDt":"2022-07-27","salesAmt":"3418297205","salesShare":"56.5","salesInten":"-445775729","salesChange":"-11.5","salesAcc":"30676698831","audiCnt":"340707","audiInten":"-45277","audiChange":"-11.7","audiAcc":"2996787","scrnCnt":"2030","showCnt":"9659"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20205362","movieNm":"\xeb\xaf\xb8\xeb\x8b\x88\xec\x96\xb8\xec\xa6\x882","openDt":"2022-07-20","salesAmt":"843554512","salesShare":"13.9","salesInten":"-60441616","salesChange":"-6.7","salesAcc":"16439332847","audiCnt":"90863","audiInten":"-6344","audiChange":"-6.5","audiAcc":"1693081","scrnCnt":"945","showCnt":"2837"},{"r

In [5]:
# 바이트자료형임을 확인 
type(json_raw)

bytes

In [6]:
# 따라서 utf-8 형식으로 고쳐줘야 합니다. .decode()를 이용합니다.
# encode => 우리가 쓰는 문자를 컴퓨터가 쓰는 형태로 변환 
# decode => 컴퓨터가 쓰는 형태의 문자를 우리가 쓰는 형태로 변환
json_utf8 = json_raw.decode("utf-8")

In [7]:
# 딕셔너리 구조를 유지하고 있는 문자로 변환됨
type(json_utf8)

str

In [8]:
json_utf8

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20220802~20220802","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20209343","movieNm":"한산: 용의 출현","openDt":"2022-07-27","salesAmt":"3418297205","salesShare":"56.5","salesInten":"-445775729","salesChange":"-11.5","salesAcc":"30676698831","audiCnt":"340707","audiInten":"-45277","audiChange":"-11.7","audiAcc":"2996787","scrnCnt":"2030","showCnt":"9659"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20205362","movieNm":"미니언즈2","openDt":"2022-07-20","salesAmt":"843554512","salesShare":"13.9","salesInten":"-60441616","salesChange":"-6.7","salesAcc":"16439332847","audiCnt":"90863","audiInten":"-6344","audiChange":"-6.5","audiAcc":"1693081","scrnCnt":"945","showCnt":"2837"},{"rnum":"3","rank":"3","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20194376","movieNm":"탑건: 매버릭","openDt":"2022-06-22","salesAmt":"678686098","salesShare":"11.2","salesInten":"-52390711"

In [9]:
# json.loads(딕셔너리 형태의 문자열 자료)
# 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
json_complete = json.loads(json_utf8)

In [10]:
# 딕셔너리(json) 변환 완료 확인
type(json_complete)

dict

In [11]:
# json_normalize(딕셔너리 자료)
# 딕셔너리 자료를 데이터프레임 형태로 변환해줌
pd.json_normalize(json_complete) # 보통 json 데이터를 받아오면 list화 되어있는 컬럼이 하나 들어옴. 해당 list 데이터를 타켓해서 데이터 가져오기

,boxOfficeResult.boxofficeType,boxOfficeResult.showRange,boxOfficeResult.dailyBoxOfficeList
0,일별 박스오피스,20220802~20220802,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."


In [12]:
# boxOfficeResult.dailyBoxOfficeList 컬럼 내부에 딕셔너리가 여럿 묶인 리스트가 보이므로
# boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20209343,한산: 용의 출현,2022-07-27,3418297205,56.5,-445775729,-11.5,30676698831,340707,-45277,-11.7,2996787,2030,9659
1,2,2,0,OLD,20205362,미니언즈2,2022-07-20,843554512,13.9,-60441616,-6.7,16439332847,90863,-6344,-6.5,1693081,945,2837
2,3,3,0,OLD,20194376,탑건: 매버릭,2022-06-22,678686098,11.2,-52390711,-7.2,76662050293,66646,-4845,-6.8,7187512,802,2072
3,4,4,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,351444239,5.8,-19020699,-5.1,2356729812,38807,-1971,-4.8,251109,677,1440
4,5,5,0,OLD,20208446,외계+인 1부,2022-07-20,363649895,6.0,-40994799,-10.1,14968915621,36506,-3867,-9.6,1434570,712,1804
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,190209473,3.1,-155223,-0.1,17660574672,18263,43,0.2,1708584,412,752
6,7,7,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,81394059,1.3,5183642,6.8,4434849457,8088,617,8.3,427202,185,271
7,8,8,1,OLD,20211792,헌트,2022-08-10,29720000,0.5,-6703000,-18.4,93969000,4280,233,5.8,10688,17,17
8,9,9,-1,OLD,20196410,비상선언,2022-08-03,26522000,0.4,-46635300,-63.7,247169216,1681,-3204,-65.6,22329,5,7
9,10,10,0,OLD,20224662,토르: 러브 앤 썬더,2022-07-06,12040299,0.2,-151561,-1.2,29461697403,1082,1,0.1,2711863,38,51


In [13]:
# json_normalize()는 pd.DataFrame()으로 대체 가능합니다.
pd.DataFrame(json_complete)

,boxOfficeResult
boxofficeType,일별 박스오피스
dailyBoxOfficeList,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."
showRange,20220802~20220802


In [14]:
url

'https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5c6ac9b9844cd60b13a5ea12c2eede98&targetDt=20220802'

## DB에 데이터를 저장하는 이유

### API 크롤링으로 서비스 제공시 

### 사용자가 조회할때마다 매번 API서버에 요청을 넣어서 

### 데이터를 가져오면 호출횟수를 금방 소모하게 됩니다. 

### 따라서, 보통 API를 활용한 서비스는 미리 데이터를  DB에 적재해놓고 

### 사용자가 요청시 내 DB에 있는 데이터를 우선적으로 보여주고, 

### 없으면 외부서버에 요청해서 내 DB에 INSERT한 다음 다시 내 DB 데이터를 사용자에게 보여줍니다.

날짜 데이터는 28까지인 달(2월), 30일까지인 달, 31일까지인 달 들이 있어서

단순히 문자열에 1씩 더하거나 날짜처리를 하기도 어렵고

위의 처리를 다 했다고 해도 4년에 1번 윤달이 있습니다.(2월 29일)

그래서, 날짜는 전적으로 날짜데이터를 이용해 처리해야만 합니다.

파이썬에서는 datetime을 이용해 처리합니다.(자바는 java.util.Date를 써서 처리)

In [15]:
import time
import datetime  # 파이썬 날짜 자료형

In [16]:
start_datetime = datetime.datetime(2021, 8, 9) # 년 월 일 순으로 넣어줍니다.
# 형식 바꾸기
target_date = start_datetime.strftime('%Y%m%d') # 영진위 형식으로 변경   
print(target_date)
# 날짜 하루 올리기
next_datetime = start_datetime + datetime.timedelta(days=1) # 날짜 자료형을 이용하면 일수만 입력하면 월별 날짜를 알아서 계산해줌
next_datetime

20210809


datetime.datetime(2021, 8, 10, 0, 0)

In [17]:
# target_dates = ['20220805', '20220806', '20220807']
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 3일치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
target_dates = ['20220805', '20220806', '20220807']
for date in target_dates : 
    request_date = date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)

    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw =r.read()

    # 한글 형식으로 변환, str 자료형
    json_utf8 = json_raw.decode("utf-8")

    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
    json_complete = json.loads(json_utf8)

    # boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
    data = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    
    data.to_excel('movie_data%s.xlsx' % request_date)


In [18]:
# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고[pd.DataFrame()]
data0805 = pd.read_excel('data/movie_data20220805.xlsx') 
data0806 = pd.read_excel('data/movie_data20220806.xlsx') 
data0807 = pd.read_excel('data/movie_data20220807.xlsx') 
data_result = data0805.append([data0806, data0807], ignore_index=True) 
data_result

C:\Users\a\AppData\Local\Temp\ipykernel_20792\96729394.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_result = data0805.append([data0806, data0807], ignore_index=True)


,Unnamed: 0,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,0,1,1,1,OLD,20209343,한산: 용의 출현,2022-07-27,2565550184,41.6,530985656,26.1,37567514644,250627,41280,19.7,3692258,1530,6343
1,1,2,2,-1,OLD,20196410,비상선언,2022-08-03,2094794151,34.0,71338791,3.5,7693545482,213574,-8542,-3.8,794759,1718,7281
2,2,3,3,1,OLD,20194376,탑건: 매버릭,2022-06-22,504483247,8.2,148299938,41.6,77905758474,46942,12422,36.0,7305825,605,1238
3,3,4,4,-1,OLD,20205362,미니언즈2,2022-07-20,413915957,6.7,948579,0.2,17793954400,43092,-1769,-3.9,1837970,783,1595
4,4,5,5,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,167553618,2.7,-5960802,-3.4,2909201168,18088,-1181,-6.1,312147,537,779
5,5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,142017614,2.3,34703455,32.3,18014179742,13226,3017,29.6,1741846,265,432
6,6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,113278760,1.8,22370761,24.6,15283607796,11557,2267,24.4,1466378,407,576
7,7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,49992320,0.8,-4086959,-7.6,192969299,5266,-680,-11.4,20859,362,513
8,8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,27421500,0.4,3966180,16.9,4509942774,2517,232,10.2,434339,75,94
9,9,10,10,26,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,12398000,0.2,11939500,2604.0,28919650670,2433,2378,4323.6,2806159,20,20


In [19]:
print(type(data_result))

<class 'pandas.core.frame.DataFrame'>


In [20]:
# 강사님께서 진행한 풀이

# target_dates = ['20220805', '20220806', '20220807']
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 3일치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
target_dates = ['20220805', '20220806', '20220807']

merged_data = pd.DataFrame()
for date in target_dates : 
    request_date = date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)

    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw =r.read()

    # 한글 형식으로 변환, str 자료형
    json_utf8 = json_raw.decode("utf-8")

    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
    json_complete = json.loads(json_utf8)

    # boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['날짜'] = date
    
    merged_data = pd.concat([merged_data, box_result])

# 결과
merged_data


,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,1,OLD,20209343,한산: 용의 출현,2022-07-27,2565550184,41.6,530997656,26.1,37567502644,250627,41282,19.7,3692256,1530,6343,20220805
1,2,2,-1,OLD,20196410,비상선언,2022-08-03,2094794151,34.0,71360791,3.5,7693510482,213574,-8540,-3.8,794756,1718,7281,20220805
2,3,3,1,OLD,20194376,탑건: 매버릭,2022-06-22,504483247,8.2,148299938,41.6,77905702474,46942,12422,36,7305821,605,1238,20220805
3,4,4,-1,OLD,20205362,미니언즈2,2022-07-20,413915957,6.7,948579,0.2,17793920900,43092,-1769,-3.9,1837967,783,1595,20220805
4,5,5,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,167553618,2.7,-5960802,-3.4,2909201168,18088,-1181,-6.1,312147,537,779,20220805
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,142017614,2.3,34703455,32.3,18014179742,13226,3017,29.6,1741846,265,432,20220805
6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,113278760,1.8,22370761,24.6,15283607796,11557,2267,24.4,1466378,407,576,20220805
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,49992320,0.8,-4086959,-7.6,192969299,5266,-680,-11.4,20859,362,513,20220805
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,27421500,0.4,3966180,16.9,4509942774,2517,232,10.2,434339,75,94,20220805
9,10,10,26,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,12398000,0.2,11939500,2604,28919650670,2433,2378,4323.6,2806159,20,20,20220805


In [21]:
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 1년치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
start_datetime = datetime.datetime(2021, 8, 8) # 년 월 일 순으로 넣어줍니다.

# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고[pd.DataFrame()]
merged_data = pd.DataFrame()
for idx in range(365) : 
    # 날짜를 문자로 변경
    target_date = start_datetime.strftime('%Y%m%d')
    
    # 요청을 넣을 날짜
    request_date = target_date
    url = "https://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
    
    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw =r.read()

    # 한글 형식으로 변환, str 자료형
    json_utf8 = json_raw.decode("utf-8")

    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환 
    json_complete = json.loads(json_utf8)

    # boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['날짜'] = date
    
    merged_data = pd.concat([merged_data, box_result])
    
    # 해당 날짜 데이터를 가져와 통합한 뒤에는 날짜를 하루 더해주기
    start_datetime = start_datetime + datetime.timedelta(days=1) # 날짜
    print("%s일차 자료 수집 완료" % idx)
    
    # 너무 자주 크롤링시 블록먹을 가능성이 있음
    time.sleep(1)
    
# 결과
merged_data


0일차 자료 수집 완료
1일차 자료 수집 완료
2일차 자료 수집 완료
3일차 자료 수집 완료
4일차 자료 수집 완료
5일차 자료 수집 완료
6일차 자료 수집 완료
7일차 자료 수집 완료
8일차 자료 수집 완료
9일차 자료 수집 완료
10일차 자료 수집 완료
11일차 자료 수집 완료
12일차 자료 수집 완료
13일차 자료 수집 완료
14일차 자료 수집 완료
15일차 자료 수집 완료
16일차 자료 수집 완료
17일차 자료 수집 완료
18일차 자료 수집 완료
19일차 자료 수집 완료
20일차 자료 수집 완료
21일차 자료 수집 완료
22일차 자료 수집 완료
23일차 자료 수집 완료
24일차 자료 수집 완료
25일차 자료 수집 완료
26일차 자료 수집 완료
27일차 자료 수집 완료
28일차 자료 수집 완료
29일차 자료 수집 완료
30일차 자료 수집 완료
31일차 자료 수집 완료
32일차 자료 수집 완료
33일차 자료 수집 완료
34일차 자료 수집 완료
35일차 자료 수집 완료
36일차 자료 수집 완료
37일차 자료 수집 완료
38일차 자료 수집 완료
39일차 자료 수집 완료
40일차 자료 수집 완료
41일차 자료 수집 완료
42일차 자료 수집 완료
43일차 자료 수집 완료
44일차 자료 수집 완료
45일차 자료 수집 완료
46일차 자료 수집 완료
47일차 자료 수집 완료
48일차 자료 수집 완료
49일차 자료 수집 완료
50일차 자료 수집 완료
51일차 자료 수집 완료
52일차 자료 수집 완료
53일차 자료 수집 완료
54일차 자료 수집 완료
55일차 자료 수집 완료
56일차 자료 수집 완료
57일차 자료 수집 완료
58일차 자료 수집 완료
59일차 자료 수집 완료
60일차 자료 수집 완료
61일차 자료 수집 완료
62일차 자료 수집 완료
63일차 자료 수집 완료
64일차 자료 수집 완료
65일차 자료 수집 완료
66일차 자료 수집 완료
67일차 자료 수집 완료
68일차 자료 수집 완료
69일차 자료 수집 완료
70일차 자료 수집 완료
71일차 자료 수집 완료
72

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,2000829380,56.5,38859180,2,16579074530,196672,4041,2.1,1712375,1639,6757,20220807
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,676797240,19.1,-127499190,-15.9,2942253950,63095,-12197,-16.2,281377,1033,3306,20220807
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,284206020,8.0,1506480,0.5,7511398580,30046,108,0.4,836805,777,1654,20220807
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,194489780,5.5,-28197330,-12.7,29457331370,18740,-2673,-12.5,2908679,540,924,20220807
4,5,5,0,OLD,20218875,극장판 도라에몽: 진구의 신공룡,2021-08-05,88917620,2.5,-4415810,-4.7,337195080,9837,-612,-5.9,38669,502,781,20220807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,6,1,OLD,20209654,헤어질 결심,2022-06-29,169174033,1.7,-32677140,-16.2,18385204948,15203,-2798,-15.5,1775050,266,426,20220807
6,7,7,-1,OLD,20208446,외계+인 1부,2022-07-20,123965118,1.3,-98324419,-44.2,15629862451,11690,-7177,-38,1496935,374,499,20220807
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,77727199,0.8,2170399,2.9,346253298,7903,248,3.2,36417,357,466,20220807
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,44635198,0.5,623298,1.4,4598589872,4122,13,0.3,442570,91,115,20220807


In [22]:
# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요.
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다.
# 컬럼 타입 object이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다.
# 현재 수치데이터도 전부 문자로 간주되고 있기 때문에 int로 자료형을 바꿔서 계산해야함
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int) 
movie_table = merged_data.pivot_table(index='movieNm', values=['audiCnt', 'scrnCnt'], aggfunc='sum')
movie_table['screenMean'] = movie_table['audiCnt'] / movie_table['scrnCnt']
movie_table.sort_values(by='audiCnt', ascending=False)

,audiCnt,scrnCnt,screenMean
movieNm,,,
범죄도시 2,12686014,81645,155.380170
스파이더맨: 노 웨이 홈,7541536,90668,83.177483
탑건: 매버릭,7448591,57863,128.728047
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
한산: 용의 출현,4597826,22656,202.940766
...,...,...,...
지지,334,1,334.000000
포 언투 어스,300,1,300.000000
맛있는 여동생,259,20,12.950000


In [23]:
# 강사님 풀이 , 따로 나누는 이유는 내 풀이와의 차이점을 알고 다시 봤을때
# "이때 이런 실수를 했구나"를 알기 위함

# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요.
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다.
# 컬럼 타입 object이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다.
# 현재 수치데이터도 전부 문자로 간주되고 있기 때문에 int로 자료형을 바꿔서 계산해야함

# 데이터 분석에서 가장 중요한 것은 쓸 데이터를 잘 뽑아내는 것이 정말 중요함
# 원하는 데이터를 원하는 곳에 가져오는 능력이 능숙해야 적절한 데이터 분석이 가능
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int) 
audi_scrn_sum = merged_data.pivot_table(index='movieNm', values=['audiCnt', 'scrnCnt'], aggfunc='sum').sort_values(by='audiCnt', ascending=False)
audi_scrn_sum['audiPerScrn'] = audi_scrn_sum['audiCnt'] / audi_scrn_sum['scrnCnt']
audi_scrn_sum

,audiCnt,scrnCnt,audiPerScrn
movieNm,,,
범죄도시 2,12686014,81645,155.380170
스파이더맨: 노 웨이 홈,7541536,90668,83.177483
탑건: 매버릭,7448591,57863,128.728047
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
한산: 용의 출현,4597826,22656,202.940766
...,...,...,...
지지,334,1,334.000000
포 언투 어스,300,1,300.000000
맛있는 여동생,259,20,12.950000


In [29]:
# 스크린당 관객수로 정렬해 가장 효율이 좋았던 영화들을 뽑아주세요.
# 집계대상의 스크린 개수는 최소 100개로 하겠습니다.
final_data = audi_scrn_sum[audi_scrn_sum['scrnCnt'] >= 100].sort_values(by=
                'audiPerScrn', ascending=False)
final_data

,audiCnt,scrnCnt,audiPerScrn
movieNm,,,
한산: 용의 출현,4597826,22656,202.940766
비상선언,1397962,8489,164.679232
범죄도시 2,12686014,81645,155.380170
탑건: 매버릭,7448591,57863,128.728047
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
...,...,...,...
리디밍 러브,1442,224,6.437500
도어맨,2679,449,5.966592
드림걸즈,1243,212,5.863208


In [30]:
# 스크린 개수가 50개 이상인 영화 중 스크린당 관객수가 평균이상인 영화만 나열해주세요.
final_data2 = final_data[(final_data['scrnCnt'] >= 50) & (final_data['audiPerScrn'] >= 26.8)].sort_values(by=
                'audiPerScrn', ascending=False).head(50)
final_data2

,audiCnt,scrnCnt,audiPerScrn
movieNm,,,
한산: 용의 출현,4597826,22656,202.940766
비상선언,1397962,8489,164.679232
범죄도시 2,12686014,81645,155.380170
탑건: 매버릭,7448591,57863,128.728047
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
쥬라기 월드: 도미니언,2832531,24893,113.788254
마녀(魔女) Part2. The Other One,2795796,27322,102.327648
토르: 러브 앤 썬더,2711863,26516,102.272703
미니언즈2,1971475,19463,101.293480


In [31]:
# csv로 export 하기
final_data2.to_csv('평균이상영화.csv', encoding='utf-8')

In [32]:
# excel로 export하기
final_data2.to_excel('평균이상영화.xlsx', encoding="utf-8")

# 가져온 데이터 mysql에 연동하기

- import pymysql
- 을 활용해서 연동할 수 있습니다.

- 임포트가 되지 않는다면 다시 설치를 해야합니다.
1. Anaconda Navigator에서 좌측 탭의 enviroments를 누릅니다.
2. Open terminal을 이용해 cmd창 같은 창을 켭니다.(터미널창이라고 부릅니다.)
3. pip install pymysql 을 입력하고 실행합니다.

### 오라클 연동

위와 같은데 3번에서 
- pip install cx_Oracle
- 을 대신 입력해서 설치하면 됩니다.

In [33]:
# pymysql 실행을 해야 DB와 연결할 수 있습니다.
import pymysql

In [34]:
# MySQL 연결에 필요한 정보를 입력합니다.
con = pymysql.connect(host='localhost', user='root', password='1111', db='pyprac', charset='utf8',
                        cursorclass=pymysql.cursors.DictCursor) # 컬럼명까지 받아올지 여부

In [35]:
# SQL 구문을 받아주는 cursor 객체를 사용합니다.
cur = con.cursor()

In [36]:
# SQL 구문은 문자로 cur.execute()를 실행해서 수행시킵니다.
cur.execute("SELECT * FROM test_tbl2")

7

In [37]:
# execute가 실행되면, cur가 데이터를 저장하고 있는데 이걸 파이썬 형식으로 빼야합니다.
mysql_data = cur.fetchall()

In [38]:
mysql_data # 딕셔너리 형태로 전달되는 것을 확인할 수 있습니다.

[{'tno': 1, 'tname': 'a'},
 {'tno': 2, 'tname': 'b'},
 {'tno': 3, 'tname': 'c'},
 {'tno': 4, 'tname': 'd'},
 {'tno': 5, 'tname': 'e'},
 {'tno': 6, 'tname': 'f'},
 {'tno': 7, 'tname': 'g'}]

In [39]:
# 데이터프레임 형태로 변환 
pd.DataFrame(mysql_data)

,tno,tname
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,f
6,7,g


In [40]:
# 여러분들이 data 'g'를 넣어보세요.
cur.execute("INSERT INTO test_tbl2(tname) VALUES(null, 'g')")

OperationalError: (1136, "Column count doesn't match value count at row 1")

In [ ]:
# 커서가 아닌 커넥션에 commit을 주는 것에 주의해주세요.
con.commit()

In [ ]:
# 다시 select 구문을 날려서 DB에 추가되었는지 확인까지 해주세요
cur.execute("SELECT * FROM test_tbl2")

In [ ]:
result_data = cur.fetchall()
pd.DataFrame(result_data)

In [41]:
merged_data.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,날짜
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,2000829380,56.5,38859180,2,16579074530,196672,4041,2.1,1712375,1639,6757,20220807
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,676797240,19.1,-127499190,-15.9,2942253950,63095,-12197,-16.2,281377,1033,3306,20220807
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,284206020,8.0,1506480,0.5,7511398580,30046,108,0.4,836805,777,1654,20220807
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,194489780,5.5,-28197330,-12.7,29457331370,18740,-2673,-12.5,2908679,540,924,20220807
4,5,5,0,OLD,20218875,극장판 도라에몽: 진구의 신공룡,2021-08-05,88917620,2.5,-4415810,-4.7,337195080,9837,-612,-5.9,38669,502,781,20220807


In [42]:
# 로우개수
len(merged_data) # 3650번 반복해야 DB에 insert가 된다는걸 알 수 있음
# iloc[], loc[] 를 이용하면 특정 로우자료를 조회할 수 있고, 거기서 ['컬럼명']을 집어넣으면
# 특정 로우의 특정 컬럼값을 조회할 수 있습니다.
merged_data.iloc[4]['movieNm']

'극장판 도라에몽: 진구의 신공룡'

In [43]:
# 과제, MySQL이나 Oracle SQL에 현재 보고있는 db_insert데이터를
# 테이블 생성 및 반복문을 이용해서 다 DB에 넣어주시면 됩니다.
# 반복문을 이용해서 merged_data의 모든 데이터를 MySQL에 집어넣어주세요.
# 집어넣을 컬럼은 rnum, movieNm, openDt, audiCnt, showCnt, date
merged_data['rnum'] = merged_data['rnum'].astype(int)
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int) 
db_insert_data = merged_data[['rnum', 'movieNm', 'openDt', 'audiCnt', 'showCnt', '날짜']] # 데이터를 다음과 같이 정제시킨 것은 매우 좋았음
db_insert_data.iloc[0]['openDt'] # DataFrame형태의 데이터

'2021-07-28'

In [61]:
# 반복문을 통해 DB에 3650개의 데이터 넣기
for i in range(len(db_insert_data)):
    data = db_insert_data.iloc[i]
    
    # 각 데이터 가져오기
    movieNm = data['movieNm']
    openDt = data['openDt']
    audiCnt = data['audiCnt']
    showCnt = data['showCnt']
    date = data['날짜']
    # 쿼리문 작성
    cur.execute("INSERT INTO movie_tbl(movieNm, openDt, audiCnt, showCnt, dataDate) VALUES('%s', '%s', %s, %s, '%s')" % (movieNm, openDt, audiCnt, showCnt, date))

# 커서가 아닌 커넥션에 commit을 주는 것에 주의해주세요.
con.commit()

OperationalError: (1292, "Incorrect date value: ' ' for column 'openDt' at row 1")

In [45]:
# 강사님 풀이

# 과제, MySQL이나 Oracle SQL에 현재 보고있는 db_insert데이터를
# 테이블 생성 및 반복문을 이용해서 다 DB에 넣어주시면 됩니다.
# 반복문을 이용해서 merged_data의 모든 데이터를 MySQL에 집어넣어주세요.
# 집어넣을 컬럼은 rnum, movieNm, openDt, audiCnt, showCnt, date

In [47]:
cur.execute("INSERT INTO movie_tbl(movieNm, openDt, audiCnt, showCnt, dataDate) VALUES('%s', '%s', %s, %s, '%s')" % ("테스트", '2022-08-10', 300, 1, '20220810'))

1

In [48]:
con.commit()

In [82]:
# merged_data변수 내부의 영화 제목만 한 줄 한 줄 콘솔에 찍어보세요.
# len() 을 이용해 데이터프레임의 로우 개수를 구할 수 있습니다.
range(len(merged_data))
# 위 range를 이용해 iloc[번호]로 로우를 순서대로 조회가능합니다.
for i in range(len(merged_data)):      
    selected_data = merged_data.iloc[i]
    print((selected_data['movieNm']))
    # 나머지(개봉일, 관객수, 상영수, 데이터 날짜)
    print((selected_data['openDt']))
    print((selected_data['audiCnt']))
    print((selected_data['showCnt']))
    print((selected_data['날짜']))
    print("--------------------------------------")


모가디슈
2021-07-28
196672
6757
20220807
--------------------------------------
더 수어사이드 스쿼드
2021-08-04
63095
3306
20220807
--------------------------------------
보스 베이비 2
2021-07-21
30046
1654
20220807
--------------------------------------
블랙 위도우
2021-07-07
18740
924
20220807
--------------------------------------
극장판 도라에몽: 진구의 신공룡
2021-08-05
9837
781
20220807
--------------------------------------
정글 크루즈
2021-07-28
9825
704
20220807
--------------------------------------
방법: 재차의
2021-07-28
5122
427
20220807
--------------------------------------
블랙핑크 더 무비
2021-08-04
3081
194
20220807
--------------------------------------
그린 나이트
2021-08-05
1855
234
20220807
--------------------------------------
랑종
2021-07-14
1508
105
20220807
--------------------------------------
모가디슈
2021-07-28
75348
6247
20220807
--------------------------------------
더 수어사이드 스쿼드
2021-08-04
24477
3007
20220807
--------------------------------------
보스 베이비 2
2021-07-21
9547
1315
20220807
------------------------------

1303
20220807
--------------------------------------
리스펙트
2021-09-08
6005
60
20220807
--------------------------------------
맨 인 더 다크 2
2021-09-01
3700
514
20220807
--------------------------------------
코다
2021-08-31
2248
366
20220807
--------------------------------------
보이스
2021-09-15
1866
10
20220807
--------------------------------------
기적
2021-09-15
1594
10
20220807
--------------------------------------
귀문
2021-08-25
1026
126
20220807
--------------------------------------
샹치와 텐 링즈의 전설
2021-09-01
43484
5731
20220807
--------------------------------------
인질
2021-08-18
15039
2334
20220807
--------------------------------------
모가디슈
2021-07-28
13792
1881
20220807
--------------------------------------
싱크홀
2021-08-11
6104
1272
20220807
--------------------------------------
맨 인 더 다크 2
2021-09-01
3851
529
20220807
--------------------------------------
보이스
2021-09-15
2449
21
20220807
--------------------------------------
코다
2021-08-31
2208
364
20220807
---------------------------

화이트데이: 부서진 결계
2021-10-06
1238
391
20220807
--------------------------------------
007 노 타임 투 다이
2021-09-29
81909
5572
20220807
--------------------------------------
보이스
2021-09-15
31648
2470
20220807
--------------------------------------
기적
2021-09-15
15774
1506
20220807
--------------------------------------
극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들
2021-09-15
5574
385
20220807
--------------------------------------
용과 주근깨 공주
2021-09-29
4817
379
20220807
--------------------------------------
샹치와 텐 링즈의 전설
2021-09-01
4362
396
20220807
--------------------------------------
극장판 포켓몬스터: 정글의 아이, 코코
2021-09-15
4221
320
20220807
--------------------------------------
그래비티
2013-10-17
2683
315
20220807
--------------------------------------
스틸워터
2021-10-06
2662
535
20220807
--------------------------------------
모가디슈
2021-07-28
2498
278
20220807
--------------------------------------
007 노 타임 투 다이
2021-09-29
95990
5591
20220807
--------------------------------------
보이스
2021-09-15
37003
2537
202208

2021-09-15
655
152
20220807
--------------------------------------
싱크홀
2021-08-11
520
2
20220807
--------------------------------------
빌리 홀리데이
2021-11-04
433
87
20220807
--------------------------------------
이터널스
2021-11-03
84299
9046
20220807
--------------------------------------
듄
2021-10-20
18998
1628
20220807
--------------------------------------
베놈 2: 렛 데어 비 카니지
2021-10-13
6271
1060
20220807
--------------------------------------
고장난 론
2021-10-27
2767
446
20220807
--------------------------------------
디어 에반 핸슨
2021-11-17
1300
9
20220807
--------------------------------------
보이스
2021-09-15
939
178
20220807
--------------------------------------
강릉
2021-11-10
795
7
20220807
--------------------------------------
기적
2021-09-15
772
111
20220807
--------------------------------------
세버그
2021-11-04
653
116
20220807
--------------------------------------
아네트
2021-10-27
596
87
20220807
--------------------------------------
이터널스
2021-11-03
89556
7054
20220807
----------------------

--------------------------------------
태일이
2021-12-01
2412
174
20220807
--------------------------------------
연애 빠진 로맨스
2021-11-24
18387
2595
20220807
--------------------------------------
유체이탈자
2021-11-24
17760
3183
20220807
--------------------------------------
엔칸토: 마법의 세계
2021-11-24
10875
1467
20220807
--------------------------------------
돈 룩 업
2021-12-08
8024
1745
20220807
--------------------------------------
듄
2021-10-20
5603
594
20220807
--------------------------------------
고스트버스터즈 라이즈
2021-12-01
2744
607
20220807
--------------------------------------
몬스타엑스 : 더 드리밍
2021-12-08
2628
214
20220807
--------------------------------------
극장판 소드 아트 온라인 -프로그레시브- 별 없는 밤의 아리아
2021-12-09
2611
228
20220807
--------------------------------------
이터널스
2021-11-03
2074
417
20220807
--------------------------------------
마이 뉴욕 다이어리
2021-12-09
2051
272
20220807
--------------------------------------
유체이탈자
2021-11-24
28789
3538
20220807
--------------------------------------
연애 빠진 로맨스
202

...

--------------------------------------
레벤느망
2022-03-10
1068
94
20220807
--------------------------------------
나이트메어 앨리
2022-02-23
707
62
20220807
--------------------------------------
이상한 나라의 수학자
2022-03-09
61987
5249
20220807
--------------------------------------
더 배트맨
2022-03-01
42401
4031
20220807
--------------------------------------
극장판 주술회전 0
2022-02-17
11154
1154
20220807
--------------------------------------
언차티드
2022-02-16
5599
819
20220807
--------------------------------------
나의 히어로 아카데미아 더 무비: 월드 히어로즈 미션
2022-03-09
5399
728
20220807
--------------------------------------
블랙라이트
2022-03-09
4017
871
20220807
--------------------------------------
안테벨룸
2022-02-23
1928
145
20220807
--------------------------------------
스파이더맨: 노 웨이 홈
2021-12-15
1557
137
20220807
--------------------------------------
극장판 바다 탐험대 옥토넛 : 해저동굴 대탈출	
2022-02-17
957
97
20220807
--------------------------------------
인민을 위해 복무하라
2022-02-23
906
98
20220807
--------------------------------------
이상한 

--------------------------------------
공기살인
2022-04-22
1000
7
20220807
--------------------------------------
뜨거운 피
2022-03-23
928
214
20220807
--------------------------------------
말임씨를 부탁해
2022-04-13
807
229
20220807
--------------------------------------
신비한 동물들과 덤블도어의 비밀
2022-04-13
50841
6875
20220807
--------------------------------------
극장판 주술회전 0
2022-02-17
6339
281
20220807
--------------------------------------
모비우스
2022-03-30
2849
878
20220807
--------------------------------------
스텔라
2022-04-06
2830
783
20220807
--------------------------------------
앰뷸런스
2022-04-06
2352
829
20220807
--------------------------------------
수퍼 소닉2
2022-04-06
2275
665
20220807
--------------------------------------
공기살인
2022-04-22
1000
7
20220807
--------------------------------------
하빗
2022-04-14
1000
50
20220807
--------------------------------------
말임씨를 부탁해
2022-04-13
862
230
20220807
--------------------------------------
루이스 웨인: 사랑을 그린 고양이 화가
2022-04-06
791
236
20220807
--------------

403
20220807
--------------------------------------
민스미트 작전
2022-05-11
3919
777
20220807
--------------------------------------
서울괴담
2022-04-27
3582
201
20220807
--------------------------------------
엄마
2022-05-11
3448
343
20220807
--------------------------------------
어부바
2022-05-11
2153
395
20220807
--------------------------------------
닥터 스트레인지: 대혼돈의 멀티버스
2022-05-04
81382
9786
20220807
--------------------------------------
니 부모 얼굴이 보고 싶다
2022-04-27
3694
748
20220807
--------------------------------------
배드 가이즈
2022-05-04
2700
681
20220807
--------------------------------------
민스미트 작전
2022-05-11
2219
688
20220807
--------------------------------------
신비한 동물들과 덤블도어의 비밀
2022-04-13
2007
423
20220807
--------------------------------------
어부바
2022-05-11
1329
368
20220807
--------------------------------------
엄마
2022-05-11
884
279
20220807
--------------------------------------
서울괴담
2022-04-27
653
156
20220807
--------------------------------------
극장판 엉덩이 탐정: 수플레 섬의 비밀
2022-05-05

1040
42
20220807
--------------------------------------
탑건: 매버릭
2022-06-22
1031
3
20220807
--------------------------------------
룸 쉐어링
2022-06-22
998
5
20220807
--------------------------------------
경아의 딸
2022-06-16
940
78
20220807
--------------------------------------
극장판 포켓몬스터DP: 기라티나와 하늘의 꽃다발 쉐이미
2022-06-01
825
90
20220807
--------------------------------------
마녀(魔女) Part2. The Other One
2022-06-15
224648
6633
20220807
--------------------------------------
범죄도시 2
2022-05-18
118477
4568
20220807
--------------------------------------
브로커
2022-06-08
37326
2290
20220807
--------------------------------------
버즈 라이트이어
2022-06-15
21994
2485
20220807
--------------------------------------
쥬라기 월드: 도미니언
2022-06-01
13161
783
20220807
--------------------------------------
모비우스
2022-03-30
1368
9
20220807
--------------------------------------
실종
2022-06-15
1306
92
20220807
--------------------------------------
앵커
2022-04-20
1110
3
20220807
--------------------------------------
극장판 포켓몬스

범죄도시 2
2022-05-18
6382
491
20220807
--------------------------------------
엘비스
2022-07-13
6226
1034
20220807
--------------------------------------
더 킬러: 죽어도 되는 아이
2022-07-13
4221
496
20220807
--------------------------------------
외계+인 1부
2022-07-20
3695
17
20220807
--------------------------------------
뒤틀린 집
2022-07-13
2252
227
20220807
--------------------------------------
그레이 맨
2022-07-13
2093
343
20220807
--------------------------------------
탑건: 매버릭
2022-06-22
100087
5517
20220807
--------------------------------------
토르: 러브 앤 썬더
2022-07-06
38827
4106
20220807
--------------------------------------
헤어질 결심
2022-06-29
37297
2156
20220807
--------------------------------------
명탐정 코난: 할로윈의 신부
2022-07-13
24580
1463
20220807
--------------------------------------
엘비스
2022-07-13
7197
1075
20220807
--------------------------------------
범죄도시 2
2022-05-18
6233
505
20220807
--------------------------------------
더 킬러: 죽어도 되는 아이
2022-07-13
4129
495
20220807
----------------------------

In [102]:
# 위의 반복문 코드를 응용해 전체 데이터를 다 DB에 넣고 확인해주세요.

for i in range(len(merged_data)):      
    selected_row = merged_data.iloc[i]
    movieNm = selected_row['movieNm']
    # 나머지(개봉일, 관객수, 상영수, 데이터 날짜)
    openDt = selected_row['openDt']
    audiCnt = selected_row['audiCnt']
    showCnt = selected_row['showCnt']
    datadate = selected_row['날짜']
    cur.execute("INSERT INTO movie_tbl(movieNm, openDt, audiCnt, showCnt, dataDate) VALUES('%s', '%s', %s, %s, '%s')" % (selected_row['movieNm'], selected_row['openDt'], selected_row['audiCnt'], selected_row['showCnt'], selected_row['날짜']))
con.commit()    

OperationalError: (1292, "Incorrect date value: ' ' for column 'openDt' at row 1")

In [103]:
# 날짜형으로 바꾸고 싶은 컬럼을 pd.to_datetime()에 넣어주면 변환됩니다.
merged_data['openDt'] = merged_data['openDt'].astype('datetime64[ns]')

ParserError: String does not contain a date:  

In [86]:
# SELECT 구문 날려서 DataFrame으로 다시 받아오기
